In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers biopython kaggle
print("✅ Đã cài đặt xong thư viện!")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 53.9 MB/s eta 0:00:00
✅ Đã cài đặt xong thư viện!


In [ ]:

import os
from google.colab import files

# Upload file kaggle.json
print("Vui lòng upload file kaggle.json của bạn:")
files.upload()

# Cấu hình Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Tải dữ liệu cuộc thi (Sẽ mất khoảng 1-2 phút)
print("⏳ Đang tải dữ liệu CAFA 6...")
!kaggle competitions download -c cafa-6-protein-function-prediction
!unzip -q cafa-6-protein-function-prediction.zip -d /content/cafa6_data
print("✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data")

Vui lòng upload file kaggle.json của bạn:


Saving kaggle.json to kaggle.json
⏳ Đang tải dữ liệu CAFA 6...
 99% 90.0M/91.3M [00:00<00:00, 938MB/s]
100% 91.3M/91.3M [00:00<00:00, 896MB/s]
✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data


In [ ]:
import torch
import numpy as np
from tqdm.auto import tqdm

def calculate_weighted_fmax(y_true, y_pred_probs, terms_list, ia_dict, thresholds=None, device='cuda'):
    """
    Tính Weighted F-max SIÊU TỐC trên GPU.
    """

    # 1. Chuyển dữ liệu sang GPU (Nếu chưa có)
    if not isinstance(y_true, torch.Tensor):
        y_true = torch.tensor(y_true, dtype=torch.float32, device=device)
    else:
        y_true = y_true.float().to(device)

    if not isinstance(y_pred_probs, torch.Tensor):
        y_pred_probs = torch.tensor(y_pred_probs, dtype=torch.float32, device=device)
    else:
        y_pred_probs = y_pred_probs.to(device)

    # 2. Chuẩn bị Vector Trọng số (IA Weights)
    # weights shape: [N_classes] -> [N_classes, 1] để broadcast
    weights_np = np.array([ia_dict.get(term, 0.0) for term in terms_list], dtype=np.float32)
    weights = torch.tensor(weights_np, dtype=torch.float32, device=device)

    # 3. Pre-calculate Mẫu số của Recall (Ground Truth Weights)
    # Dùng phép nhân ma trận: [N_samples, N_classes] @ [N_classes] -> [N_samples]
    # Đây là tổng trọng số các nhãn đúng của mỗi protein
    total_true_weight = torch.matmul(y_true, weights)

    # Lọc bỏ các protein không có nhãn thật nào (để tránh chia cho 0)
    valid_mask = total_true_weight > 0
    y_true_valid = y_true[valid_mask]
    y_pred_valid = y_pred_probs[valid_mask]
    total_true_weight_valid = total_true_weight[valid_mask]

    n_samples = y_true_valid.shape[0]
    if n_samples == 0:
        return 0.0, 0.0, 0.0, 0.0

    # 4. Định nghĩa ngưỡng
    if thresholds is None:
        thresholds = torch.arange(0.01, 1.01, 0.01, device=device)
    else:
        thresholds = torch.tensor(thresholds, device=device)

    fmax_scores = []
    precisions = []
    recalls = []

    # 5. Loop qua từng ngưỡng (Vẫn cần loop nhưng phép tính bên trong cực nhanh)
    # Tắt tqdm leave=False để đỡ rác màn hình
    for t in tqdm(thresholds, desc="GPU Scanning", leave=False):

        # Binarize: [N, C] (Tạo mask 0/1)
        pred_binary = (y_pred_valid >= t).float()

        # --- TÍNH TOÁN BẰNG MA TRẬN (MATRIX MULTIPLICATION) ---

        # 1. Tổng trọng số dự đoán (Predicted Weights) = Pred * W
        # [N, C] @ [C] -> [N]
        pred_total_weights = torch.matmul(pred_binary, weights)

        # 2. Trọng số giao thoa (Intersection Weights) = (Pred * True) * W
        # Element-wise mul trước để tìm True Positive, sau đó nhân ma trận với weights
        intersection_weights = torch.matmul(pred_binary * y_true_valid, weights)

        # --- Tính Precision ---
        # Xử lý chia cho 0: Nếu pred_total_weights = 0 thì precision = 0
        precision_sample = torch.zeros_like(intersection_weights)
        has_pred_mask = pred_total_weights > 0
        precision_sample[has_pred_mask] = intersection_weights[has_pred_mask] / pred_total_weights[has_pred_mask]

        avg_precision = precision_sample.mean().item()

        # --- Tính Recall ---
        # Mẫu số total_true_weight_valid luôn > 0 (đã lọc ở trên)
        recall_sample = intersection_weights / total_true_weight_valid
        avg_recall = recall_sample.mean().item()

        # --- Tính F1 ---
        if avg_precision + avg_recall > 0:
            f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
        else:
            f1 = 0.0

        fmax_scores.append(f1)
        precisions.append(avg_precision)
        recalls.append(avg_recall)

    # 6. Tìm Max
    # Chuyển về CPU để xử lý index
    fmax_scores = np.array(fmax_scores)
    best_idx = np.argmax(fmax_scores)

    best_fmax = fmax_scores[best_idx]
    best_threshold = thresholds[best_idx].item()

    return best_fmax, best_threshold, precisions[best_idx], recalls[best_idx]

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import gc
import os
import sys
from tqdm.auto import tqdm

# ============================================================================
# CONFIGURATION
# ============================================================================
CONFIG = {
    'EMBEDDINGS': {
        'ankh': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_test_FINAL_embeddings.npy',
        },
        # 'ankh': {
        #     'train': '/content/drive/MyDrive/CAFA6_Embeddings_2/Ankh_Large_Aligned/ankh_large_train_FINAL_embeddings.npy',
        #     'test':  '/content/drive/MyDrive/CAFA6_Embeddings_2/Ankh_Large_Aligned/ankh_large_test_FINAL_embeddings.npy',
        # },
        'esm': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh/esm2_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh/esm2_test_FINAL_embeddings.npy',
        },
        'prot_t5': {
            'train': '/content/drive/MyDrive/CAFA6_Embeddings/Prot_T5_Aligned/protT5_xl_train_FINAL_embeddings.npy',
            'test':  '/content/drive/MyDrive/CAFA6_Embeddings/Prot_T5_Aligned/protT5_xl_test_FINAL_embeddings.npy',
        },
        # 'protBERT': {
        #     'train': '/content/drive/MyDrive/CAFA6_Embeddings/protBERT_Aligned/protBERT_train_FINAL_embeddings.npy',
        #     'test':  '/content/drive/MyDrive/CAFA6_Embeddings/protBERT_Aligned/protBERT_test_FINAL_embeddings.npy',
        # },
    },

    'TRAIN_ID_PATH': '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_train_FINAL_ids.npy',
    'TEST_ID_PATH':  '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_test_FINAL_ids.npy',

    'SAVE_DIR': '/content/drive/MyDrive/CAFA6_Results/prott5_esm2_ankh_Run',
    'TRAIN_TERMS': '/content/cafa6_data/Train/train_terms.tsv',
    'IA_FILE': '/content/cafa6_data/IA.tsv',
    'TEST_FASTA': '/content/cafa6_data/Test/testsuperset.fasta',

    # Model Params
    'TOP_K_LABELS': 10000, # Giữ mức này để model học tốt
    # 'MIN_FREQ': 3,

    # Input sẽ là 768 (Ankh) + 1280 (ESM) = 2048
    'ENCODER_LAYERS': [2048, 1024, 512],

    'DROPOUT_RATE': 0.4, # Tăng dropout vì input lớn
    'EPOCHS': 80,
    'BATCH_SIZE': 128,
    'LEARNING_RATE': 5e-4, # Giảm LR một chút cho ổn định
    'LABEL_SMOOTHING': 0.1,

    # Inference Params (Chuẩn 0.27 điểm)
    'MIN_CONFIDENCE': 0.01,
    'MAX_PREDS_PER_PROTEIN': 20,

    'SEED': 42,
    'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu'
}

os.makedirs(CONFIG['SAVE_DIR'], exist_ok=True)
print(f"🚀 CAFA 6 - DUAL MODEL (ANKH + ESM) | Device: {CONFIG['DEVICE']}")

torch.manual_seed(CONFIG['SEED'])
np.random.seed(CONFIG['SEED'])

# ============================================================================
# 1. MEMORY-SAFE DATASET (CHÌA KHÓA ĐỂ KHÔNG TRÀN RAM)
# ============================================================================
class MultiSourceDataset(Dataset):
    def __init__(self, embedding_paths_dict, y_tensor=None, indices=None):
        self.mmaps = {}
        self.keys = list(embedding_paths_dict.keys())

        # Load mmap
        for name, path in embedding_paths_dict.items():
            self.mmaps[name] = np.load(path, mmap_mode='r')

        # Base length
        first_key = self.keys[0]
        self.total_len = len(self.mmaps[first_key])

        self.indices = indices if indices is not None else np.arange(self.total_len)
        self.y = y_tensor

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        real_idx = self.indices[idx]

        inputs = []
        # Load từng vector -> List of Tensors
        for key in self.keys:
            vec = torch.from_numpy(self.mmaps[key][real_idx].copy()).float()
            inputs.append(vec)

        if self.y is not None:
            return inputs, self.y[real_idx]
        return (inputs,)

# ============================================================================
# 2. LOAD DATA & PROCESS LABELS
# ============================================================================
print("\n[1/5] Checking Files...")
train_ids = np.load(CONFIG['TRAIN_ID_PATH'])
print(f"   ✓ Train IDs: {len(train_ids)}")

print("\n[2/5] Processing Labels (IA Strategy)...")
# 1. Load Terms & IA
df_terms = pd.read_csv(CONFIG['TRAIN_TERMS'], sep='\t', header=0, names=['EntryID', 'term', 'aspect'])
df_ia = pd.read_csv(CONFIG['IA_FILE'], sep='\t', names=['term', 'ia'])
ia_dict = dict(zip(df_ia['term'], df_ia['ia']))

# 2. Tính Score
term_counts = df_terms['term'].value_counts().reset_index()
term_counts.columns = ['term', 'freq']

# print(f"   Original terms count: {len(term_counts)}")
# term_counts = term_counts[term_counts['freq'] >= CONFIG['MIN_FREQ']]
# print(f"   -> After filtering (Freq >= {CONFIG['MIN_FREQ']}): {len(term_counts)} terms")

term_counts['ia'] = term_counts['term'].map(ia_dict).fillna(0.0)
term_counts['score'] = term_counts['freq'] * term_counts['ia']

# 3. Chọn Top K
top_terms_df = term_counts.sort_values(by='score', ascending=False).head(CONFIG['TOP_K_LABELS'])
top_terms = top_terms_df['term'].tolist()
print(f"   ✓ Selected {len(top_terms)} terms.")

# 4. Map ID -> Terms
df_filtered = df_terms[df_terms['term'].isin(top_terms)]
id_to_terms = df_filtered.groupby('EntryID')['term'].apply(list).to_dict()

# Dọn dẹp
del df_terms, df_filtered, df_ia, term_counts, top_terms_df
gc.collect()

# ============================================================================
# 3. PREPARE LABELS (RAM OPTIMIZED)
# ============================================================================
print("\n[3/5] Preparing Labels (Sparse Mode)...")

# 1. Sparse MLB
mlb = MultiLabelBinarizer(classes=top_terms, sparse_output=True)
mlb.fit([top_terms])

# 2. Transform -> Sparse Matrix
y_labels_list = [id_to_terms.get(pid, []) for pid in train_ids]
y_train_sparse = mlb.transform(y_labels_list)
del y_labels_list, train_ids # Xóa ID list không dùng nữa
gc.collect()

# 3. Weights (Optional)
# Nếu dùng IA weight thì giữ, nếu muốn model tự học thì comment dòng này và bỏ pos_weight trong Loss
weights_list = [ia_dict.get(t, 0.0) for t in mlb.classes_]
pos_weight_tensor = torch.tensor(weights_list, dtype=torch.float32).to(CONFIG['DEVICE'])

# 4. Convert to Dense Float32 & Label Smoothing
print("   ⏳ Converting Labels to Tensor...")
# Convert từng phần nhỏ hoặc convert hết nếu RAM > 12GB (với 80k row x 10k col float32 ~ 3.2GB -> Ổn)
y_train_binary = y_train_sparse.astype(np.float32).toarray()
y_train_tensor = torch.from_numpy(y_train_binary)

if CONFIG['LABEL_SMOOTHING'] > 0:
    y_train_tensor.mul_(1 - CONFIG['LABEL_SMOOTHING']).add_(CONFIG['LABEL_SMOOTHING'] / len(top_terms))

del y_train_sparse, y_train_binary
gc.collect()

# ============================================================================
# 4. CREATE DATA LOADERS
# ============================================================================
print("\n[4/5] Creating DataLoaders...")

train_paths = {k: v['train'] for k, v in CONFIG['EMBEDDINGS'].items()}

indices = np.arange(len(y_train_tensor))
train_idx, val_idx = train_test_split(indices, test_size=0.15, random_state=CONFIG['SEED'])

train_dataset = MultiSourceDataset(train_paths, y_train_tensor, indices=train_idx)
val_dataset   = MultiSourceDataset(train_paths, y_train_tensor, indices=val_idx)

# --- AUTO DETECT INPUT DIMS ---
print("   ⏳ Measuring input shapes from dataset...")
sample_inputs, _ = train_dataset[0]
INPUT_DIMS_LIST = [x.shape[0] for x in sample_inputs]
print(f"   ✓ Detected Input Dims: {INPUT_DIMS_LIST}")

train_loader = DataLoader(train_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=False, num_workers=2)

# ============================================================================
# 5. MODEL & TRAINING
# ============================================================================
print("\n[5/5] Building Model...")

class EncoderBlock(nn.Module):
    def __init__(self, in_dim, layers_config, dropout):
        super().__init__()
        layers = []

        # 1. LAYER NORM ĐẦU VÀO (BẮT BUỘC)
        # Để cân bằng 'âm lượng' giữa ESM (hét to) và Ankh (nói nhỏ)
        layers.append(nn.LayerNorm(in_dim))

        prev_dim = in_dim
        for i, dim in enumerate(layers_config):
            layers.extend([
                nn.Linear(prev_dim, dim),
                # nn.LayerNorm(dim), # Ổn định training
                nn.GELU(),           # Hiện đại hơn ReLU
                nn.Dropout(dropout)
            ])
            prev_dim = dim

        self.net = nn.Sequential(*layers)
        self.out_dim = prev_dim

    def forward(self, x):
        return self.net(x)

# Module lọc nhiễu (Attention)
class SEBlock(nn.Module):
    """Squeeze-and-Excitation Block để lọc nhiễu sau khi gộp"""
    def __init__(self, in_dim, reduction=4):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_dim, in_dim // reduction, bias=False),
            nn.ReLU(),
            nn.Linear(in_dim // reduction, in_dim, bias=False),
            nn.Sigmoid() # Tạo ra mask từ 0 đến 1
        )

    def forward(self, x):
        # x shape: [Batch, Dim]
        # Attention weight: [Batch, Dim]
        w = self.fc(x)
        # Nhân trọng số vào x: Cái nào quan trọng thì giữ, rác thì nhân với 0
        return x * w

class MultiModalNet(nn.Module):
    def __init__(self, input_dims_list, encoder_layers, dropout, num_classes):
        super().__init__()

        self.encoders = nn.ModuleList()
        self.fusion_input_dim = 0

        print("\n🏗️ Building Advanced Architecture:")

        # 1. Xây dựng các nhánh Encoder
        for i, in_dim in enumerate(input_dims_list):
            print(f"   ➤ Branch {i+1}: Input {in_dim} -> Output {encoder_layers[-1]}")
            enc = EncoderBlock(in_dim, encoder_layers, dropout)
            self.encoders.append(enc)
            self.fusion_input_dim += enc.out_dim

        print(f"   ➤ Fusion Dim: {self.fusion_input_dim}")

        # 2. SE-Block (Bộ lọc thông minh)
        self.attention_filter = SEBlock(self.fusion_input_dim)

        # 3. Layer tổng hợp cuối cùng
        self.head = nn.Sequential(
            # nn.LayerNorm(self.fusion_input_dim),
            nn.Linear(self.fusion_input_dim, 512),
            nn.GELU(),
            nn.Dropout(dropout),

            # Thêm một lớp nữa để tăng khả năng học
            nn.Linear(512, 512),
            nn.GELU(),
            nn.Dropout(dropout),

            nn.Linear(512, num_classes)
        )

    def forward(self, inputs_list):
        features = []
        # Đi qua từng nhánh
        for i, encoder in enumerate(self.encoders):
            feat = encoder(inputs_list[i])
            features.append(feat)

        # Gộp lại (Concatenate)
        combined = torch.cat(features, dim=1)

        # LỌC NHIỄU (Điểm khác biệt lớn nhất)
        # Mạng sẽ tự học cách "tắt tiếng" các đặc trưng rác từ ProtT5 nếu nó thấy không cần thiết
        refined = self.attention_filter(combined)

        return self.head(refined)

model = MultiModalNet(
    input_dims_list = INPUT_DIMS_LIST,
    encoder_layers  = CONFIG['ENCODER_LAYERS'],
    dropout         = CONFIG['DROPOUT_RATE'],
    num_classes     = len(top_terms) # Dùng số lượng thực tế sau lọc
).to(CONFIG['DEVICE'])

# Loss & Optimizer
# Có thể thử bỏ pos_weight nếu thấy loss dao động quá mạnh
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
optimizer = optim.AdamW(model.parameters(), lr=CONFIG['LEARNING_RATE'], weight_decay=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
# scheduler = optim.lr_scheduler.OneCycleLR(
#     optimizer,
#     max_lr=1e-3,              # LR cao nhất có thể đạt được (thường gấp 10 lần LR khởi điểm)
#     epochs=CONFIG['EPOCHS'],  # Tổng số epoch
#     steps_per_epoch=len(train_loader),
#     pct_start=0.3,            # 30% thời gian đầu dùng để tăng tốc (Warmup)
#     div_factor=25,            # LR khởi điểm = max_lr / 25
#     final_div_factor=1000     # LR kết thúc = LR khởi điểm / 1000
# )

print(f"\n🚀 START TRAINING ({CONFIG['EPOCHS']} Epochs)...")
best_val_loss = float('inf')
best_fmax = 0.0
best_thresh = 0.0
best_loss_model_path = f"{CONFIG['SAVE_DIR']}/best_loss_model.pth"
best_f1_model_path = f"{CONFIG['SAVE_DIR']}/best_f1_model.pth"
best_train_f1_model_path = f"{CONFIG['SAVE_DIR']}/best_train_f1_model.pth"
last_epoch_model_path = f"{CONFIG['SAVE_DIR']}/last_epoch_model.pth"

for epoch in range(CONFIG['EPOCHS']):
    model.train()
    train_loss = 0
    for X_b, y_b in train_loader:
        X_b, y_b = [x.to(CONFIG['DEVICE']) for x in X_b], y_b.to(CONFIG['DEVICE'])
        optimizer.zero_grad()
        logits = model(X_b)
        loss = criterion(logits, y_b)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        # scheduler.step()
        train_loss += loss.item()

    avg_train = train_loss / len(train_loader)

    model.eval()
    val_loss = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for X_b, y_b in val_loader:
            X_b, y_b = [x.to(CONFIG['DEVICE']) for x in X_b], y_b.to(CONFIG['DEVICE'])
            logits = model(X_b)
            loss = criterion(logits, y_b)
            val_loss += loss.item()
            probs = torch.sigmoid(logits)

            all_preds.append(probs)
            all_targets.append(y_b)

    avg_val = val_loss / len(val_loader)
    scheduler.step(avg_val)

    all_preds_tensor = torch.cat(all_preds, dim=0)
    all_targets_tensor = torch.cat(all_targets, dim=0)

    print("  ⏳ Calculating Weighted F-max...")

    # Gọi hàm tính toán
    w_fmax, w_thresh, w_prec, w_rec = calculate_weighted_fmax(
        y_true=all_targets_tensor,
        y_pred_probs=all_preds_tensor,
        terms_list=top_terms,  # List 10,000 term ID
        ia_dict=ia_dict,        # Dict {TermID: IA} bạn đã load
        # thresholds=np.arange(0.01, 1.01, 0.05)
    )

    print(f"Epoch {epoch+1:02d} | Train: {avg_train:.4f} | Val: {avg_val:.4f} | LR: {optimizer.param_groups[0]['lr']:.1e} | wF_max: {w_fmax:.4f} (Thresh: {w_thresh:.2f})")

    if w_fmax > best_fmax:  # Nhớ khai báo best_fmax = 0.0 ở ngoài loop
        best_fmax = w_fmax
        best_thesh = w_thresh
        torch.save(model.state_dict(), best_f1_model_path)
        print(f"  🏆 Saved Best Weighted F-max Model! ({best_fmax:.4f})")

    if avg_val < best_val_loss:
        best_val_loss = avg_val
        torch.save(model.state_dict(), best_loss_model_path)
        print("  ⭐ New Best Model!")

torch.save(model.state_dict(), last_epoch_model_path)
print(f"  🏆 Saved last epoch model")

# ============================================================================
# SWA
# ============================================================================
# Code gộp 3 checkpoint tốt nhất để tạo model ổn định
# def average_models(paths):
#     avg_state = None
#     for path in paths:
#         state = torch.load(path)
#         if avg_state is None:
#             avg_state = state
#         else:
#             for k in avg_state.keys():
#                 avg_state[k] += state[k]

#     # Chia trung bình
#     for k in avg_state.keys():
#         avg_state[k] = avg_state[k] / len(paths)
#     return avg_state

# paths = [
#     best_loss_model_path,
#     best_f1_model_path
# ]

# swa_model_path = f"{CONFIG['SAVE_DIR']}/swa_model.pth"

# swa_state = average_models(paths)
# torch.save(swa_state, swa_model_path)
# print("✅ Đã tạo SWA Model ổn định!")

# ============================================================================
# 6. INFERENCE (STREAMING)
# ============================================================================
print("\n🔮 PREDICTING...")

# Dọn dẹp Training
del train_loader, val_loader, train_dataset, val_dataset, y_train_tensor, optimizer
gc.collect()
torch.cuda.empty_cache()

# model.load_state_dict(torch.load(best_f1_model_path))
# model.load_state_dict(torch.load(last_epoch_model_path))
model.load_state_dict(torch.load(best_loss_model_path))
# model.load_state_dict(torch.load(swa_model_path))
model.eval()

# Load Test IDs & Paths
test_paths = {k: v['test'] for k, v in CONFIG['EMBEDDINGS'].items()}
test_ids = np.load(CONFIG['TEST_ID_PATH'])

# Test Dataset (Dual Memmap)
test_dataset = MultiSourceDataset(test_paths)
test_loader = DataLoader(test_dataset, batch_size=CONFIG['BATCH_SIZE']*2, shuffle=False, num_workers=2)

submission_path = f"{CONFIG['SAVE_DIR']}/submission.tsv"
n_predictions = 0

with open(submission_path, 'w') as f:
    current_idx = 0
    with torch.no_grad():
        for (X_b,) in tqdm(test_loader, desc="Inference"):
            X_b = [x.to(CONFIG['DEVICE']) for x in X_b]
            logits = model(X_b)
            probs_batch = torch.sigmoid(logits).cpu().numpy()

            ids_batch = test_ids[current_idx : current_idx + len(probs_batch)]
            current_idx += len(probs_batch)

            for i, pid in enumerate(ids_batch):
                probs = probs_batch[i]

                # TOP-K CỨNG (Chiến thuật 0.27 điểm)
                top_k = CONFIG['MAX_PREDS_PER_PROTEIN']
                ind = np.argpartition(probs, -top_k)[-top_k:]
                ind = ind[np.argsort(probs[ind])][::-1]

                for idx in ind:
                    score = probs[idx]
                    if score > CONFIG['MIN_CONFIDENCE']:
                        f.write(f"{pid}\t{top_terms[idx]}\t{score:.3f}\n")
                        n_predictions += 1

            del probs_batch, X_b, logits

print(f"\n✅ DONE! File: {submission_path}")
print(f"\n✅ DONE! Predictions: {n_predictions:,}")

🚀 CAFA 6 - DUAL MODEL (ANKH + ESM) | Device: cuda

[1/5] Checking Files...
   ✓ Train IDs: 82404

[2/5] Processing Labels (IA Strategy)...
   ✓ Selected 10000 terms.

[3/5] Preparing Labels (Sparse Mode)...
   ⏳ Converting Labels to Tensor...

[4/5] Creating DataLoaders...
   ⏳ Measuring input shapes from dataset...
   ✓ Detected Input Dims: [768, 1280, 1024]

[5/5] Building Model...

🏗️ Building Advanced Architecture:
   ➤ Branch 1: Input 768 -> Output 512
   ➤ Branch 2: Input 1280 -> Output 512
   ➤ Branch 3: Input 1024 -> Output 512
   ➤ Fusion Dim: 1536

🚀 START TRAINING (80 Epochs)...
  ⏳ Calculating Weighted F-max...


GPU Scanning:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch 01 | Train: 0.0160 | Val: 0.0059 | LR: 5.0e-04 | wF_max: 0.1069 (Thresh: 0.09)
  🏆 Saved Best Weighted F-max Model! (0.1069)
  ⭐ New Best Model!
  ⏳ Calculating Weighted F-max...


GPU Scanning:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch 02 | Train: 0.0060 | Val: 0.0057 | LR: 5.0e-04 | wF_max: 0.1129 (Thresh: 0.11)
  🏆 Saved Best Weighted F-max Model! (0.1129)
  ⭐ New Best Model!
  ⏳ Calculating Weighted F-max...


...

Epoch 79 | Train: 0.0032 | Val: 0.0041 | LR: 3.9e-06 | wF_max: 0.2215 (Thresh: 0.20)
  ⏳ Calculating Weighted F-max...


GPU Scanning:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch 80 | Train: 0.0032 | Val: 0.0041 | LR: 3.9e-06 | wF_max: 0.2216 (Thresh: 0.21)
  🏆 Saved last epoch model

🔮 PREDICTING...


Inference:   0%|          | 0/877 [00:00<?, ?it/s]


✅ DONE! File: /content/drive/MyDrive/CAFA6_Results/prott5_esm2_ankh_Run/submission.tsv

✅ DONE! Predictions: 4,310,351


In [ ]:
# Chạy lệnh này trong một cell mới
!kaggle competitions submit \
    -c cafa-6-protein-function-prediction \
    -f /content/drive/MyDrive/CAFA6_Results/prott5_esm2_ankh_Run/submission.tsv \
    -m "esm2 ankh prot t5 best_loss_model_path no norm"

100% 99.0M/99.0M [00:01<00:00, 54.0MB/s]
Successfully submitted to CAFA 6 Protein Function Prediction

In [ ]:
!kaggle competitions submissions -c cafa-6-protein-function-prediction

fileName        date                        description                                                                              status                     publicScore  privateScore  
--------------  --------------------------  ---------------------------------------------------------------------------------------  -------------------------  -----------  ------------  
submission.tsv  2025-12-13 08:51:37.060000  esm2 ankh prot t5 best_loss_model_path no norm                                           SubmissionStatus.COMPLETE  0.262                      
submission.tsv  2025-12-13 08:07:33.953000  esm2 ankh prot t5 best_loss_model_path no norm                                           SubmissionStatus.COMPLETE  0.251                      
submission.tsv  2025-12-13 07:41:23.780000  esm2 ankh prot t5 best_loss_model_path layernorm                                         SubmissionStatus.COMPLETE  0.257                      
submission.tsv  2025-12-13 05:07:03.243000  esm2 ankh prot t